# BERT

In [1]:
#pip install optuna

In [2]:
import sys
sys.path.append('..')

import import_ipynb
from data_preparation import Preprocessing

importing Jupyter notebook from /home/g.russo55/Progetto/src/classifiers/../data_preparation/Preprocessing.ipynb


In [3]:
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np

import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

In [4]:
df = pd.read_csv('../../data/updated_tweets.csv')

### Normalize

In [5]:
def normalize_tweet_BERT(tweet):
    tweet = Preprocessing.remove_links_mentions(tweet)
    tweet = tweet.lower()
    tweet = Preprocessing.remove_hashtag(tweet)
    tweet = Preprocessing.remove_special_characters(tweet)
 
    tweet = Preprocessing.remove_spaces(tweet)
    tweet = Preprocessing.remove_textual_emojis(tweet)
    tweet = Preprocessing.remove_not_ASCII(tweet)

    return tweet

In [6]:
df['tweet_text'] = df['tweet_text'].apply(normalize_tweet_BERT)
df = Preprocessing.clean_normalized_df(df)

In [7]:
df['cyberbullying_type'].value_counts()

cyberbullying_type
religion             7963
age                  7949
ethnicity            7893
not_cyberbullying    7711
gender               7665
Name: count, dtype: int64

In seguito a questo risultato, confermiamo che i dati sono bilanciati

### Labels encoding

In [8]:
possible_labels = df.cyberbullying_type.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'not_cyberbullying': 0, 'gender': 1, 'religion': 2, 'age': 3, 'ethnicity': 4}

Sostituiamo nel dataset

In [9]:
df['label'] = df.cyberbullying_type.replace(label_dict)
df.head(3)

/tmp/ipykernel_629571/349901490.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df.cyberbullying_type.replace(label_dict)


,tweet_text,cyberbullying_type,label
0,"in other words katandandre, your food was crap...",not_cyberbullying,0
1,why is aussietv so white?,not_cyberbullying,0
2,a classy whore? or more red velvet cupcakes?,not_cyberbullying,0


### Dataset split

In [10]:
from sklearn.model_selection import train_test_split

# Divido il dataset in training e validation set mantenendo la distribuzione delle classi
'''
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=None)

# Creare una colonna per segnare il tipo di dato
df['data_type'] = ['not_set']*df.shape[0]

# Assegna 'train' ai dati di addestramento e 'val' ai dati di validazione nel DataFrame
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

# Conta i valori per ogni combinazione di categoria, etichetta e tipo di dato
df.groupby(['cyberbullying_type', 'label', 'data_type']).count()

from sklearn.model_selection import train_test_split, GridSearchCV
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np

# Splitta il dataset in train+validation e test
X_train_val, X_test, y_train_val, y_test = train_test_split(df.tweet_text, df.label, test_size=0.2, random_state=42)

# Splitta ulteriormente train+validation in train e validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2
'''

"\nX_train, X_val, y_train, y_val = train_test_split(df.index.values, \n                                                  df.label.values, \n                                                  test_size=0.15, \n                                                  random_state=42, \n                                                  stratify=None)\n\n# Creare una colonna per segnare il tipo di dato\ndf['data_type'] = ['not_set']*df.shape[0]\n\n# Assegna 'train' ai dati di addestramento e 'val' ai dati di validazione nel DataFrame\ndf.loc[X_train, 'data_type'] = 'train'\ndf.loc[X_val, 'data_type'] = 'val'\n\n# Conta i valori per ogni combinazione di categoria, etichetta e tipo di dato\ndf.groupby(['cyberbullying_type', 'label', 'data_type']).count()\n\nfrom sklearn.model_selection import train_test_split, GridSearchCV\nfrom transformers import BertForSequenceClassification, Trainer, TrainingArguments\nimport numpy as np\n\n# Splitta il dataset in train+validation e test\nX_train_val, X_test, y

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import optuna
import numpy as np
from datasets import load_metric


# Suddividi il dataset in train, validation e test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# Converti i dataframe in dataset di Hugging Face
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenizza il testo
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['tweet_text'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Imposta il formato per i dataset
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Definisci la funzione obiettivo per Optuna
def compute_metrics(p):
    metric = load_metric("accuracy")
    predictions, labels = p
    preds = np.argmax(predictions, axis=1)
    return metric.compute(predictions=preds, references=labels)

def objective(trial):
    # Definisci i parametri da ottimizzare
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32])
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 5)
    warmup_steps = trial.suggest_int("warmup_steps", 0, 200)

    # Configura gli argomenti di training
    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        warmup_steps=warmup_steps,
        evaluation_strategy="epoch",
        logging_dir='./logs',
        logging_steps=10
    )

    # Definisci il modello
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

    # Definisci il Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    # Addestra il modello
    trainer.train()

    # Valuta il modello
    eval_result = trainer.evaluate()
    return eval_result['eval_accuracy']

# Crea lo studio di Optuna e ottimizza
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# Migliori parametri
print("Best hyperparameters: ", study.best_params)

# Valutazione del modello migliore sul test set
best_trial = study.best_trial
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_trial.params["learning_rate"],
    per_device_train_batch_size=best_trial.params["batch_size"],
    num_train_epochs=best_trial.params["num_train_epochs"],
    warmup_steps=best_trial.params["warmup_steps"],
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10
)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

# Valutazione sul test set
test_result = trainer.evaluate(test_dataset)
print("Test set results: ", test_result)


Map:   0%|          | 0/23508 [00:00<?, ? examples/s]

KeyboardInterrupt: 